<a href="https://colab.research.google.com/github/120Davies/DS-Unit-2-Regression-Classification/blob/master/Copy_of_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`) using a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do exploratory visualizations with Seaborn.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a linear regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).
(That book is good regardless of whether your cultural worldview is inferential statistics or predictive machine learning)
- [ ] Read Leo Breiman's paper, ["Statistical Modeling: The Two Cultures"](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

In [30]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [33]:
df = df[(df['SALE_PRICE'] >= 100000) &
        (df['SALE_PRICE'] <= 2000000) &
        (df['GROSS_SQUARE_FEET'] <= 19000)]
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OCEAN PARKWAY-NORTH,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
65,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,NaN,R4,"505 WEST 19TH STREET, 7D",7D,10011.0,1.0,0.0,1.0,"18,975",1077.0,2013.0,2,R4,2000000,01/02/2019
66,1,MIDTOWN EAST,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,NaN,D4,"345 EAST 54TH ST, 3B",NaN,10022.0,0.0,0.0,0.0,0,0.0,1960.0,2,D4,330000,01/02/2019
67,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,NaN,D4,"16 EAST 80TH STREET, 2A",NaN,10075.0,0.0,0.0,0.0,0,0.0,1925.0,2,D4,600000,01/02/2019


In [34]:
# Test/train split

import numpy as np
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format = True)
df['SALE_DATE'].dt.month.value_counts()

1    3709
3    3548
2    3180
4    2946
Name: SALE_DATE, dtype: int64

In [35]:
# 3581 from March for training, 2983 from April for testing
train = df[df.SALE_DATE.dt.month == 3]
test = df[df.SALE_DATE.dt.month == 4]
train.shape, test.shape

((3548, 21), (2946, 21))

In [36]:
# Test/train split check

import plotly.express as px
px.scatter(train, x='GROSS_SQUARE_FEET', y='SALE_PRICE')

In [37]:
train.groupby('BOROUGH').SALE_PRICE.mean()

BOROUGH
1    958695.660828
2    491024.178571
3    832023.584137
4    634863.362179
5    557381.824207
Name: SALE_PRICE, dtype: float64

In [38]:
# Significiant correlations there
train.groupby('BOROUGH').YEAR_BUILT.describe()

,count,mean,std,min,25%,50%,75%,max
BOROUGH,,,,,,,,
1,628.0,1788.587580,544.307828,0.0,1923.0,1952.0,1965.25,2017.0
2,392.0,1764.941327,562.654441,0.0,1920.0,1931.0,1958.25,2018.0
3,931.0,1807.211600,508.987907,0.0,1910.0,1931.0,1970.00,2018.0
4,1248.0,1881.402244,365.152241,0.0,1930.0,1949.0,1960.00,2018.0
5,347.0,1900.469741,361.561300,0.0,1940.0,1970.0,1990.00,2018.0


In [39]:
# too many 0's
train['YEAR_BUILT'].value_counts()

1920.0    233
0.0       214
1930.0    200
1925.0    177
1960.0    151
1950.0    142
1910.0    125
1940.0    122
1955.0     89
2015.0     79
1965.0     73
1935.0     71
1901.0     70
1931.0     67
1945.0     67
2016.0     65
2017.0     60
1899.0     56
1963.0     54
1952.0     54
1915.0     47
1970.0     45
2007.0     44
1964.0     41
1951.0     40
1962.0     39
2005.0     38
2018.0     35
1900.0     31
2004.0     31
         ... 
1981.0      5
2012.0      5
1982.0      5
1933.0      5
1917.0      4
1992.0      4
1912.0      4
1997.0      4
1947.0      4
1971.0      4
1974.0      3
1908.0      3
1946.0      3
1936.0      3
1978.0      3
1906.0      3
1907.0      3
1880.0      2
1890.0      2
1909.0      2
1895.0      1
1800.0      1
1850.0      1
1902.0      1
1904.0      1
1977.0      1
1944.0      1
1918.0      1
1881.0      1
1903.0      1
Name: YEAR_BUILT, Length: 121, dtype: int64

In [40]:
# It's gettin' one-hot in herre
train['RESIDENTIAL_UNITS'].value_counts()

1.0     1475
0.0     1115
2.0      683
3.0      196
4.0       47
6.0       15
5.0        7
8.0        3
7.0        2
35.0       1
19.0       1
9.0        1
11.0       1
12.0       1
Name: RESIDENTIAL_UNITS, dtype: int64

In [41]:
# A lot of these features are just straight garbage. Apartment number can't be important.
# Other features have too many NaN's to be useful.
train['TAX_CLASS_AT_TIME_OF_SALE'].value_counts()

1    1774
2    1623
4     151
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [45]:
dum_train = pd.get_dummies(train['TAX_CLASS_AT_TIME_OF_SALE'])
dum_test = pd.get_dummies(test['TAX_CLASS_AT_TIME_OF_SALE'])
train = pd.concat([train, dum_train], axis=1)
test = pd.concat([test, dum_test], axis=1)
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,1,2,4
12062,1,CHELSEA,09 COOPS - WALKUP APARTMENTS,2,719,36,NaN,C6,"405 WEST 21ST STREET, 2R",NaN,10011.0,0.0,0.0,0.0,0,0.0,1900.0,2,C6,750000,2019-03-01,0,1,0
12063,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,766,1152,NaN,R4,"270 WEST 17TH, 4K",4K,10011.0,1.0,0.0,1.0,0,585.0,1989.0,2,R4,945000,2019-03-01,0,1,0
12069,1,GREENWICH VILLAGE-CENTRAL,13 CONDOS - ELEVATOR APARTMENTS,2,531,1059,NaN,R4,"14 EAST 4TH STREET, 825",825,10012.0,1.0,0.0,1.0,"15,272",1092.0,1912.0,2,R4,1325000,2019-03-01,0,1,0
12070,1,GREENWICH VILLAGE-WEST,10 COOPS - ELEVATOR APARTMENTS,2,626,12,NaN,D4,"61 JANE STREET, 18B",NaN,10014.0,0.0,0.0,0.0,0,0.0,1963.0,2,D4,1595000,2019-03-01,0,1,0
12072,1,HARLEM-CENTRAL,13 CONDOS - ELEVATOR APARTMENTS,2,1599,1423,NaN,R4,"40 WEST 116 STREET, B805",B805,10026.0,1.0,0.0,1.0,0,1022.0,2006.0,2,R4,985000,2019-03-01,0,1,0


In [0]:
# Tax class feature has been one-hot encoded to the features '1', '2', and '4'.
# Use SelectKBest
from sklearn.preprocessing import StandardScaler

target = 'SALE_PRICE'
high_cardinality = ['YEAR_BUILT', 'BUILDING_CLASS_CATEGORY', 'NEIGHBORHOOD', 'BLOCK']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

for k in range(1, len(X_train_encoded.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')